In [5]:
!pip uninstall -y langchain-openai

Found existing installation: langchain-openai 0.0.8
Uninstalling langchain-openai-0.0.8:
  Successfully uninstalled langchain-openai-0.0.8


In [ ]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
!pip install PyMuPDF==1.24.0
!pip install chromadb==0.4.24
!pip install pyngrok==7.1.5
!pip install pydantic==1.10.13
!pip install langchain-chroma==0.1.4

  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.39.1 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
google-adk 1.21.0 requires tenacity<10

  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached protobuf-6.33.4-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 97.8 MB/s eta 0:00:00
Using cached packaging-25.0-py3-none-any.whl (66 kB)
Using cached protobuf-6.33.4-cp39-abi3-manylinux2014_x86_64.whl (323 kB)
  Created wheel for chroma-hnswlib: filename=chroma_hnswlib-0.7.3-cp312-cp312-linux_x86_64.whl size=2529916 sha256=69ed3feb92f26796d233fe126f992d6a2a3ae42f6f85f08abbba82c14e43f0bd
  Stored in directory: /root/.cache/pip/wheels/6d/14/b5/68c4f2e056600c0348a94efba92dc975686ab72b714e0ca3d6
Successfully built chroma-hnswlib
  Attempting uninstall: protobuf
    Found

  Attempting uninstall: pyngrok
    Found existing installation: pyngrok 7.5.0
    Uninstalling pyngrok-7.5.0:
      Successfully uninstalled pyngrok-7.5.0
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
^C


In [ ]:
!pip install pyngrok

In [31]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

In [32]:
from google.colab import userdata
import os

key = userdata.get("OPENAI_API_KEY")
assert key is not None, "OPENAI_API_KEY not found in Colab Secrets"

os.environ["OPENAI_API_KEY"] = key


In [33]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["NGROK_AUTH_TOKEN"] = userdata.get("NGROK_AUTH_TOKEN")

assert os.environ["OPENAI_API_KEY"] is not None
assert os.environ["NGROK_AUTH_TOKEN"] is not None

print("Secrets loaded successfully")


Secrets loaded successfully


In [34]:
%%writefile app.py

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores.chroma import Chroma
from langchain_chroma import Chroma
from operator import itemgetter
import pandas as pd
import streamlit as st
import chromadb
import tempfile
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

# Customize initial app landing page
st.set_page_config(page_title="File QA Chatbot", page_icon="🤖")
st.title("Welcome to File QA RAG Chatbot 🤖")

@st.cache_resource(ttl="1h")
# Takes uploaded PDFs, creates document chunks, computes embeddings
# Stores document chunks and embeddings in a Vector DB
# Returns a retriever which can look up the Vector DB
# to return documents based on user input
# Stores this in the cache
def configure_retriever(uploaded_files):
  # Read documents
  docs = []
  temp_dir = tempfile.TemporaryDirectory()
  for file in uploaded_files:
    temp_filepath = os.path.join(temp_dir.name, file.name)
    with open(temp_filepath, "wb") as f:
      f.write(file.getvalue())
    loader = PyMuPDFLoader(temp_filepath)
    docs.extend(loader.load())

  # Split into documents chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                                 chunk_overlap=200)
  doc_chunks = text_splitter.split_documents(docs)
  # Add unique IDs to each document chunk
  ids = [f"doc_{i}" for i in range(len(doc_chunks))]


  # Create document embeddings and store in Vector DB
  embeddings_model = HuggingFaceEmbeddings(
      model_name="sentence-transformers/all-MiniLM-L6-v2"
  )

  client = chromadb.Client()

  # Use Chroma from LangChain
  vectordb = Chroma.from_documents(
    documents=doc_chunks,
    embedding=embeddings_model,
    ids=ids,
    client=client,
    collection_name="document_collection"
  )


  # Define retriever object
  retriever = vectordb.as_retriever(search_kwargs={"k": 3})
  return retriever

# Manages live updates to a Streamlit app's display by appending new text tokens
# to an existing text stream and rendering the updated text in Markdown
class StreamHandler(BaseCallbackHandler):
  def __init__(self, container, initial_text=""):
    self.container = container
    self.text = initial_text

  def on_llm_new_token(self, token: str, **kwargs) -> None:
    self.text += token
    self.container.markdown(self.text)

# Creates UI element to accept PDF uploads
uploaded_files = st.sidebar.file_uploader(
  label="Upload PDF files", type=["pdf"],
  accept_multiple_files=True
)
if not uploaded_files:
  st.info("Please upload PDF documents to continue.")
  st.stop()

# Create retriever object based on uploaded PDFs
retriever = configure_retriever(uploaded_files)

# Load a connection to ChatGPT LLM
hf_pipeline = pipeline(
    "text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256
)
chatgpt = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a prompt template for QA RAG System
qa_template = """
              Use only the following pieces of context to answer the question at the end.
              If you don't know the answer, just say that you don't know,
              don't try to make up an answer. Keep the answer as concise as possible.

              {context}

              Question: {question}
              """
qa_prompt = ChatPromptTemplate.from_template(qa_template)

# This function formats retrieved documents before sending to LLM
def format_docs(docs):
  return "\n\n".join([d.page_content for d in docs])

# Create a QA RAG System Chain
qa_rag_chain = (
  {
    "context": itemgetter("question") # based on the user question get context docs
      |
    retriever
      |
    format_docs,
    "question": itemgetter("question") # user question
  }
    |
  qa_prompt # prompt with above user question and context
    |
  chatgpt # above prompt is sent to the LLM for response
)

# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory(key="langchain_messages")

# Shows the first message when app starts
if len(streamlit_msg_history.messages) == 0:
  streamlit_msg_history.add_ai_message("Please ask your question?")

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# Callback handler which does some post-processing on the LLM response
# Used to post the top 3 document sources used by the LLM in RAG response
class PostMessageHandler(BaseCallbackHandler):
  def __init__(self, msg: st.write):
    BaseCallbackHandler.__init__(self)
    self.msg = msg
    self.sources = []

  def on_retriever_end(self, documents, *, run_id, parent_run_id, **kwargs):
    source_ids = []
    for d in documents: # retrieved documents from retriever based on user query
      metadata = {
        "source": d.metadata["source"],
        "page": d.metadata["page"],
        "content": d.page_content[:200]
      }
      idx = (metadata["source"], metadata["page"])
      if idx not in source_ids: # store unique source documents
        source_ids.append(idx)
        self.sources.append(metadata)

  def on_llm_end(self, response, *, run_id, parent_run_id, **kwargs):
    if len(self.sources):
      st.markdown("__Sources:__ "+"\n")
      st.dataframe(data=pd.DataFrame(self.sources[:3]),
                    width=1000) # Top 3 sources


# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    # Initializing an empty data stream
    stream_handler = StreamHandler(st.empty())
    # UI element to write RAG sources after LLM response
    sources_container = st.write("")
    pm_handler = PostMessageHandler(sources_container)
    config = {"callbacks": [stream_handler, pm_handler]}
    # Get LLM response
    response = qa_rag_chain.invoke({"question": user_prompt},
                                    config)

Overwriting app.py


In [35]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [36]:
from pyngrok import ngrok
from google.colab import userdata

# Kill any existing tunnels
ngrok.kill()

# Load NGROK auth token from Colab Secrets
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
assert NGROK_AUTH_TOKEN is not None, "NGROK_AUTH_TOKEN not found in Colab Secrets"

# Set the auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open HTTPS tunnel to Streamlit port
ngrok_tunnel = ngrok.connect(8989)

print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://meteorically-vegetative-mario.ngrok-free.dev


In [28]:
ngrok.kill()

In [29]:
!ps -ef | grep streamlit

root       22519       1  2 09:10 ?        00:01:04 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port=8989
root       34896   22348  0 10:01 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       34898   34896  0 10:01 ?        00:00:00 grep streamlit


In [30]:
!sudo kill -9 6975

kill: (6975): No such process
